In [21]:
import numpy as np

#
# Transfer functions
#
class TransferFunctions:
    def sgm(x, Derivative=False):
        if not Derivative:
            return 1.0 / (1.0 + np.exp(-x))
        else:
            #out = sgm(x)
            out = 1.0 / (1.0 + np.exp(-x))
            return out * (1.0 - out)
    
    def linear(x, Derivative=False):
        if not Derivative:
            return x
        else:
            return 1.0
    
    def gaussian(x, Derivative=False):
        if not Derivative:
            return np.exp(-x**2)
        else:
            return -2*x*np.exp(-x**2)
    
    def tanh(x, Derivative=False):
        if not Derivative:
            return np.tanh(x)
        else:
            return 1.0 - np.tanh(x)**2
    
    def truncLinear(x, Derivative=False):
        if not Derivative:
            y = x.copy()
            y[y < 0] = 0
            return y
        else:
            return 1.0
        
#normalization function
def norm(dizi):
    Maxs = np.amax(dizi,axis=0)
    #Maxs = np.array([210,200,50])
    Mins = np.amin(dizi,axis=0)
    #Mins =np.array([100,30,30])
    A = []
    for d in dizi:
        a = []
        for i in range(d.shape[0]):
            new = (d[i]-Mins[i])/(Maxs[i]-Mins[i])
            a.append(new)
        A.append(a)
    New = np.array(A)
    return New
#
# Classes
#
class BackPropagationNetwork:
    """A back-propagation network"""
    
    #
    # Class methods
    #
    def __init__(self, layerSize, layerFunctions=None):
        """Initialize the network"""
        
        self.layerCount = 0
        self.shape = None
        self.weights = []
        self.tFuncs = []
        
        # Layer info
        self.layerCount = len(layerSize) - 1
        self.shape = layerSize
        
        if layerFunctions is None:
            lFuncs = []
            for i in range(self.layerCount):
                if i == self.layerCount - 1:
                    lFuncs.append(TransferFunctions.linear)
                else:
                    lFuncs.append(TransferFunctions.sgm)
        else:
            if len(layerSize) != len(layerFunctions):
                raise ValueError("Incompatible list of transfer functions.")
            elif layerFunctions[0] is not None:
                raise ValueError("Input layer cannot have a transfer function.")
            else:
                lFuncs = layerFunctions[1:]
        
        self.tFuncs = lFuncs
        
        # Data from last Run
        self._layerInput = []
        self._layerOutput = []
        self._previousWeightDelta = []
        
        # Create the weight arrays
        for (l1,l2) in zip(layerSize[:-1], layerSize[1:]):
            self.weights.append(np.random.normal(scale=0.01, size = (l2, l1+1)))
            self._previousWeightDelta.append(np.zeros((l2, l1+1)))
    
    #
    # Run method
    #
    def Run(self, input):
        """Run the network based on the input data"""
        
        lnCases = input.shape[0]
        
        # Clear out the previous intermediate value lists
        self._layerInput = []
        self._layerOutput = []
        
        # Run it!
        for index in range(self.layerCount):
            # Determine layer input
            if index == 0:
                layerInput = self.weights[0].dot(np.vstack([input.T, np.ones([1, lnCases])]))
            else:
                layerInput = self.weights[index].dot(np.vstack([self._layerOutput[-1], np.ones([1, lnCases])]))
            
            self._layerInput.append(layerInput)
            self._layerOutput.append(self.tFuncs[index](layerInput))
        
        return self._layerOutput[-1].T
                 
    #
    # TrainEpoch method
    #
    def TrainEpoch(self, input, target, trainingRate = 0.01, momentum = 0.7): #trainingRate = 0.02, momentum = 0.5
        """This method trains the network for one epoch"""
        
        delta = []
        lnCases = input.shape[0]
        
        # First run the network
        self.Run(input)
        
        # Calculate our deltas
        for index in reversed(range(self.layerCount)):
            if index == self.layerCount - 1:
                # Compare to the target values
                output_delta = self._layerOutput[index] - target.T
                error = np.sum(output_delta**2)
                delta.append(output_delta * self.tFuncs[index](self._layerInput[index], True))
            else:
                # Compare to the following layer's delta
                delta_pullback = self.weights[index + 1].T.dot(delta[-1])
                delta.append(delta_pullback[:-1, :] * self.tFuncs[index](self._layerInput[index], True))
            
        # Compute weight deltas
        for index in range(self.layerCount):
            delta_index = self.layerCount - 1 - index
            
            if index == 0:
                layerOutput = np.vstack([input.T, np.ones([1, lnCases])])
            else:
                layerOutput = np.vstack([self._layerOutput[index - 1], np.ones([1, self._layerOutput[index - 1].shape[1]])])
            
            curWeightDelta = np.sum(\
                                 layerOutput[None,:,:].transpose(2, 0 ,1) * delta[delta_index][None,:,:].transpose(2, 1, 0)\
                                 , axis = 0)
            
            weightDelta = trainingRate * curWeightDelta + momentum * self._previousWeightDelta[index]
            
            self.weights[index] -= weightDelta
            
            self._previousWeightDelta[index] = weightDelta
        
        return error

#
# If run as a script, create a test object
#
if __name__ == "__main__":
 
    #veri içeri alma
    from numpy import genfromtxt 
    my_data = genfromtxt('winequality-red.csv', delimiter=';',skip_header=1)

    train = my_data[:1000,:] #my_data[:1200,:]
    test = my_data[1000:,:]
    train_input = train[:,:11]
    train_target= train[:,11:]
    test_input = test[:,:11]
    test_target = test[:, 11:]
    """
    print("my_data: ",my_data.shape)
    print("train: ", train.shape)
    print("test: ", test.shape)
    print("train_input shape: ",train_input.shape)
    print("train_target shape: ",train_target.shape)
    """
    #lvTarget = norm(train_target)
    lvTarget = train_target/10
    lvInput = norm(train_input)
    #katman sayısı kadar Transfer Fonksiyonu EKLE
    lFuncs = [None, TransferFunctions.tanh, TransferFunctions.linear, TransferFunctions.sgm]
    #Katman yapısını belirle
    bpn = BackPropagationNetwork((11,15,5,1), lFuncs)
    #iterasyon sayısı ve istenilen hata oranı gir
    lnMax = 10000 #50000
    lnErr = 1e-1  #1e-6
    for i in range(lnMax+1):
        err = bpn.TrainEpoch(lvInput, lvTarget,0.01,0.7)
        if i % 5000 == 0 and i > 0: #5000
            print("Iteration {0:6d}K - Error: {1:0.6f}".format(int(i/1000), err))
        if err <= lnErr:
            print("İstenilen hata oranına ulaşıldı. Iter: {0}".format(i))
            break
        
    # Display output
    
    lvOutput = bpn.Run(lvInput)
    for i in range(train_input.shape[0]):
        print("Input: {0} Output: {1} expect: {2}".format(train_input[i], lvOutput[i],lvTarget[i]))

Iteration      5K - Error: 3.959696
Iteration     10K - Error: 3.702741
Input: [ 7.4     0.7     0.      1.9     0.076  11.     34.      0.9978  3.51
  0.56    9.4   ] Output: [0.50711922] expect: [0.5]
Input: [ 7.8     0.88    0.      2.6     0.098  25.     67.      0.9968  3.2
  0.68    9.8   ] Output: [0.53652553] expect: [0.5]
Input: [7.80e+00 7.60e-01 4.00e-02 2.30e+00 9.20e-02 1.50e+01 5.40e+01 9.97e-01
 3.26e+00 6.50e-01 9.80e+00] Output: [0.54460659] expect: [0.5]
Input: [11.2    0.28   0.56   1.9    0.075 17.    60.     0.998  3.16   0.58
  9.8  ] Output: [0.56014323] expect: [0.6]
Input: [ 7.4     0.7     0.      1.9     0.076  11.     34.      0.9978  3.51
  0.56    9.4   ] Output: [0.50711922] expect: [0.5]
Input: [ 7.4     0.66    0.      1.8     0.075  13.     40.      0.9978  3.51
  0.56    9.4   ] Output: [0.5119049] expect: [0.5]
Input: [ 7.9     0.6     0.06    1.6     0.069  15.     59.      0.9964  3.3
  0.46    9.4   ] Output: [0.51289822] expect: [0.5]
Input: [ 7.

Input: [ 9.2     0.52    1.      3.4     0.61   32.     69.      0.9996  2.74
  2.      9.4   ] Output: [0.47735252] expect: [0.4]
Input: [7.50e+00 6.00e-01 3.00e-02 1.80e+00 9.50e-02 2.50e+01 9.90e+01 9.95e-01
 3.35e+00 5.40e-01 1.01e+01] Output: [0.54106548] expect: [0.5]
Input: [7.50e+00 6.00e-01 3.00e-02 1.80e+00 9.50e-02 2.50e+01 9.90e+01 9.95e-01
 3.35e+00 5.40e-01 1.01e+01] Output: [0.54106548] expect: [0.5]
Input: [7.100e+00 4.300e-01 4.200e-01 5.500e+00 7.000e-02 2.900e+01 1.290e+02
 9.973e-01 3.420e+00 7.200e-01 1.050e+01] Output: [0.53955218] expect: [0.5]
Input: [7.100e+00 4.300e-01 4.200e-01 5.500e+00 7.100e-02 2.800e+01 1.280e+02
 9.973e-01 3.420e+00 7.100e-01 1.050e+01] Output: [0.54000416] expect: [0.5]
Input: [7.100e+00 4.300e-01 4.200e-01 5.500e+00 7.000e-02 2.900e+01 1.290e+02
 9.973e-01 3.420e+00 7.200e-01 1.050e+01] Output: [0.53955218] expect: [0.5]
Input: [7.100e+00 4.300e-01 4.200e-01 5.500e+00 7.100e-02 2.800e+01 1.280e+02
 9.973e-01 3.420e+00 7.100e-01 1.050e+

  3.31     0.57    10.4    ] Output: [0.53526148] expect: [0.4]
Input: [ 7.7      0.43     0.25     2.6      0.073   29.      63.       0.99615
  3.37     0.58    10.5    ] Output: [0.56855074] expect: [0.6]
Input: [ 8.9     0.59    0.5     2.      0.337  27.     81.      0.9964  3.04
  1.61    9.5   ] Output: [0.49433955] expect: [0.6]
Input: [ 9.      0.82    0.14    2.6     0.089   9.     23.      0.9984  3.39
  0.63    9.8   ] Output: [0.52736456] expect: [0.5]
Input: [ 7.7      0.43     0.25     2.6      0.073   29.      63.       0.99615
  3.37     0.58    10.5    ] Output: [0.56855074] expect: [0.6]
Input: [ 6.9      0.52     0.25     2.6      0.081   10.      37.       0.99685
  3.46     0.5     11.     ] Output: [0.53176117] expect: [0.5]
Input: [5.200e+00 4.800e-01 4.000e-02 1.600e+00 5.400e-02 1.900e+01 1.060e+02
 9.927e-01 3.540e+00 6.200e-01 1.220e+01] Output: [0.60965014] expect: [0.7]
Input: [ 8.       0.38     0.06     1.8      0.078   12.      49.       0.99625
  3.37 

Input: [12.2    0.48   0.54   2.6    0.085 19.    64.     1.     3.1    0.61
 10.5  ] Output: [0.56195657] expect: [0.6]
Input: [11.4     0.6     0.49    2.7     0.085  10.     41.      0.9994  3.15
  0.63   10.5   ] Output: [0.5561485] expect: [0.6]
Input: [ 7.7     0.69    0.05    2.7     0.075  15.     27.      0.9974  3.26
  0.61    9.1   ] Output: [0.53145581] expect: [0.5]
Input: [ 8.7     0.31    0.46    1.4     0.059  11.     25.      0.9966  3.36
  0.76   10.1   ] Output: [0.60678124] expect: [0.6]
Input: [ 9.8     0.44    0.47    2.5     0.063   9.     28.      0.9981  3.24
  0.65   10.8   ] Output: [0.59678793] expect: [0.6]
Input: [12.      0.39    0.66    3.      0.093  12.     30.      0.9996  3.18
  0.63   10.8   ] Output: [0.60682427] expect: [0.7]
Input: [10.4    0.34   0.58   3.7    0.174  6.    16.     0.997  3.19   0.7
 11.3  ] Output: [0.62316405] expect: [0.6]
Input: [12.5     0.46    0.49    4.5     0.07   26.     49.      0.9981  3.05
  0.57    9.6   ] Output: [

Input: [8.300e+00 5.400e-01 2.400e-01 3.400e+00 7.600e-02 1.600e+01 1.120e+02
 9.976e-01 3.270e+00 6.100e-01 9.400e+00] Output: [0.51504785] expect: [0.5]
Input: [8.200e+00 5.600e-01 2.300e-01 3.400e+00 7.800e-02 1.400e+01 1.040e+02
 9.976e-01 3.280e+00 6.200e-01 9.400e+00] Output: [0.51681935] expect: [0.5]
Input: [10.      0.58    0.22    1.9     0.08    9.     32.      0.9974  3.13
  0.55    9.5   ] Output: [0.53455625] expect: [0.5]
Input: [ 7.9     0.51    0.25    2.9     0.077  21.     45.      0.9974  3.49
  0.96   12.1   ] Output: [0.65192932] expect: [0.6]
Input: [ 6.8     0.69    0.      5.6     0.124  21.     58.      0.9997  3.46
  0.72   10.2   ] Output: [0.56774941] expect: [0.5]
Input: [ 6.8     0.69    0.      5.6     0.124  21.     58.      0.9997  3.46
  0.72   10.2   ] Output: [0.56774941] expect: [0.5]
Input: [ 8.8     0.6     0.29    2.2     0.098   5.     15.      0.9988  3.36
  0.49    9.1   ] Output: [0.46942384] expect: [0.5]
Input: [ 8.8     0.6     0.29    2.

Input: [ 7.1      0.46     0.14     2.8      0.076   15.      37.       0.99624
  3.36     0.49    10.7    ] Output: [0.56506355] expect: [0.5]
Input: [ 7.5     0.27    0.34    2.3     0.05    4.      8.      0.9951  3.4
  0.64   11.    ] Output: [0.62419153] expect: [0.7]
Input: [ 7.1      0.46     0.14     2.8      0.076   15.      37.       0.99624
  3.36     0.49    10.7    ] Output: [0.56506355] expect: [0.5]
Input: [ 7.8      0.57     0.09     2.3      0.065   34.      45.       0.99417
  3.46     0.74    12.7    ] Output: [0.66272427] expect: [0.8]
Input: [ 5.9      0.61     0.08     2.1      0.071   16.      24.       0.99376
  3.56     0.77    11.1    ] Output: [0.62054824] expect: [0.6]
Input: [ 7.5      0.685    0.07     2.5      0.058    5.       9.       0.99632
  3.38     0.55    10.9    ] Output: [0.54550219] expect: [0.4]
Input: [ 5.9      0.61     0.08     2.1      0.071   16.      24.       0.99376
  3.56     0.77    11.1    ] Output: [0.62054824] expect: [0.6]
Input:

In [19]:
def Sor(a):
    girdi = np.array([a])
    print(girdi,'')
    inp = norm(girdi)
    print(inp,'')
    deger = bpn.Run(inp)

    return deger